# Activation Patching on GPT2, M1, M2 and M3 models using nnsight
- Developed on Google Colab using an A100 with 40GB GPU and 80GB system RAM.
- Runs with GPT2/TinyStories/Qwen/Llama with base/CS1/CS2/CS3.  
- Requires a GITHUB_TOKEN secret to access Martian quanta_text_to_sql code repository.
- Requires a HF_TOKEN secret to access Martian HuggingFace repository.


# Import libraries
Imports standard libraries. Do not read.

In [ ]:
# https://nnsight.net/
!pip install -U nnsight -q

In [ ]:
from IPython.display import clear_output
import einops
import torch
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"

import nnsight
from nnsight import LanguageModel, util

In [ ]:
!pip install datasets -q

In [ ]:
try:
    from getpass import getpass
    from google.colab import userdata
    import gc
    import weakref
    github_token = userdata.get("GITHUB_TOKEN")
except:
    import os
    github_token = os.getenv("GITHUB_TOKEN")

# # Install the private repository using the token
!pip install --upgrade git+https://{github_token}@github.com/withmartian/quanta_text_to_sql.git -q

import TinySQL as qts

# Select model and command set


In [ ]:
model_num = 3   # 0=GPT2, 1=TinyStories, 2=Qwen or 3=Llama
cs_num = 1      # 0=BaseModel, 1=CS1, 2=CS2 or 3=CS3

# Load Model

In [ ]:
if model_num == 1:
    the_tokenizer, the_model = qts.load_sql_interp_model(model_num, cs_num, auth_token=userdata.get("HF_TOKEN"))

    model = LanguageModel(the_model, the_tokenizer)
    model.tokenizer = the_tokenizer
else:
    if torch.backends.mps.is_available():
        model = LanguageModel(qts.sql_interp_model_location(model_num, cs_num), device_map="mps")
    else:
        model = LanguageModel(qts.sql_interp_model_location(model_num, cs_num), device_map="auto")

clear_output()
print(model)

# Run prompts to see which one the model gets right

In [ ]:
prompts = [
("### Instruction: How much do employees earn? ### Context: CREATE TABLE employees (name TEXT, salary INTEGER, department TEXT) ### Response: SELECT", 'salary'),
("### Instruction: How much does each product cost? ### Context: CREATE TABLE products (name TEXT, price INTEGER, category TEXT) ### Response: SELECT", 'price'),
("### Instruction: How tall are all the buildings? ### Context: CREATE TABLE buildings (address TEXT, height INTEGER, year_built INTEGER) ### Response: SELECT", 'height'),
("### Instruction: How many inhabitants live in each city? ### Context: CREATE TABLE cities (name TEXT, population INTEGER, country TEXT) ### Response: SELECT", 'population'),
("### Instruction: How heavy are the packages? ### Context: CREATE TABLE shipments (tracking_id TEXT, weight DECIMAL, destination TEXT) ### Response: SELECT", 'weight'),
("### Instruction: When was each book released? ### Context: CREATE TABLE books (title TEXT, publication_date DATE, author TEXT) ### Response: SELECT", 'publication_date'),
("### Instruction: How far is each destination? ### Context: CREATE TABLE locations (place TEXT, distance INTEGER, transport_mode TEXT) ### Response: SELECT", 'distance'),
("### Instruction: What's the temperature in each room? ### Context: CREATE TABLE sensors (room_id TEXT, temp_celsius DECIMAL, humidity INTEGER) ### Response: SELECT", 'temp_celsius'),
("### Instruction: How deep are the wells? ### Context: CREATE TABLE wells (location TEXT, depth INTEGER, status TEXT) ### Response: SELECT", 'depth'),
("### Instruction: How long are the movies? ### Context: CREATE TABLE films (title TEXT, duration INTEGER, genre TEXT) ### Response: SELECT", 'duration'),
("### Instruction: How fast can each vehicle go? ### Context: CREATE TABLE vehicles (model TEXT, speed INTEGER, manufacturer TEXT) ### Response: SELECT", 'speed'),
]

results = []
prob_layers_all = {}
for i, (prompt, gt) in enumerate(prompts):
    print(f"Prompt {i+1}")
    with model.generate(prompt, max_new_tokens=1, temperature=0.0001) as tracer:
        out = model.generator.output.save()

    output_text = model.tokenizer.decode(out[0], skip_special_tokens=True)
    answer = output_text.split("SELECT")[-1].strip()

    correct = answer in gt
    results.append(correct)

    if correct:
        layers = model.model.layers
        # What I want to do next is see how the probabilities change as we move through the layers
        prob_layers = []
        final_ln = model.model.norm
        lm_head = model.lm_head
        with model.trace() as tracer:
            with tracer.invoke(prompt) as invoker:
                for layer_idx, layer in enumerate(layers):
                    # Process layer output through the model's head and layer normalization
                    layer_output = lm_head(final_ln(layer.output[0]))
                    # Apply softmax to obtain probabilities and save the result
                    probs = torch.nn.functional.softmax(layer_output, dim=-1)

                    answer = " " + gt
                    answer_idx = model.tokenizer(answer)["input_ids"][1]
                    # Get the probability of the correct answer
                    correct_prob = probs[0, -1, answer_idx].save()
                    prob_layers.append(correct_prob)

        prob_layers_all[answer] = prob_layers


print(sum(results)/len(results)) # About 50-60% accuracy in this unscientific test

In [ ]:
correct_prompts = {}
for i, (prompt, gt) in enumerate(prompts):
    if results[i]:
        correct_prompts[gt] = prompt

for gt, prompt in correct_prompts.items():
    print(gt, prompt)

# See how probability of correct prompts evolve
They become higher in the last 6 layers!

In [ ]:
# set plotly to notebook
pio.renderers.default = "vscode"

# plot all on the same graph
for answer, probs in prob_layers_all.items():
    probs = [p.item() for p in probs]
    # Print probs but to 2 decimal places)
    print([f"{p:.2f}" for p in probs], answer)

# Run attention blocking

In [ ]:
attn_blocking_full_results = {}

for gt, prompt in correct_prompts.items():
    print(f"Doing {gt}")
    prompt_tokens = model.tokenizer.batch_decode(model.tokenizer(prompt)['input_ids'])
    num_tokens = len(prompt_tokens)
    num_layers = model.config.num_hidden_layers
    exp_results = {'logits': [], 'probs': [], 'tokens': prompt_tokens}

    # Get the actual indices of the last six layers
    layers_to_block = [num_layers - i - 1 for i in range(6)]

    # Block attention
    for index in range(0, num_tokens):
        indices_list = [(index, num_tokens-1)]
        with model.trace(prompt):
            # Create attention mask
            attention_mask = torch.ones(1, 1, num_tokens, num_tokens, dtype=torch.bool, device=model.device).tril(diagonal=0)
            for i, j in indices_list:
                attention_mask[:, :, j, i] = False # i think it's j, i

            # Run Blocking
            for layer_num in layers_to_block:
                attn = model.model.layers[layer_num].self_attn.inputs
                kwargs = attn[1]
                kwargs["attention_mask"] = attention_mask
                attn = (attn[0], kwargs)
                model.model.layers[layer_num].self_attn.inputs = attn

            logits = model.output.logits.save()

        # Save logits and probs to results
        exp_results['logits'].append(logits.to("cpu").detach())

        answer = " " + gt
        answer_idx = model.tokenizer(answer)["input_ids"][1]
        prob = torch.nn.functional.softmax(logits[0, -1], dim=-1)[answer_idx].item()

        exp_results['probs'].append(prob)

    attn_blocking_full_results[gt] = exp_results

In [ ]:
for gt, results in attn_blocking_full_results.items():
    title = f"Where the model is predicting '{gt}'"
    print("-"*100)
    print(f"{' '*(50- len(title)//2)}{title}")
    print("-"*100)
    for token, prob in zip(results['tokens'], results['probs']):
        if token == "<|begin_of_text|>": continue
        highlight_string = "<--- High probability drop" if prob < 0.5 else ""
        print(f"{prob:.2f} {token}   {highlight_string}")

# WIP

In [ ]:
# Do direct logit attribution
final_ln = model.model.norm
lm_head = model.lm_head

layers = model.model.layers
probs_layers = []

with model.trace() as tracer:
    with tracer.invoke(prompt) as invoker:
        for layer_idx, layer in enumerate(layers):
            # Process layer output through the model's head and layer normalization
            layer_output = lm_head(final_ln(layer.output[0]))
            # Apply softmax to obtain probabilities and save the result
            probs = torch.nn.functional.softmax(layer_output, dim=-1).save()
            probs_layers.append(probs)

probs = torch.cat([probs.value for probs in probs_layers])

# Find the maximum probability and corresponding tokens for each position
max_probs, tokens = probs.max(dim=-1)

# Decode token IDs to words for each layer
words = [[model.tokenizer.decode(t.cpu()).encode("unicode_escape").decode() for t in layer_tokens]
    for layer_tokens in tokens]

clean_tokens = model.tokenizer(prompt)["input_ids"]
input_words = [model.tokenizer.decode(t) for t in clean_tokens]
print( input_words )